特徴量の重みを正常に反映しているか確認するため、オッズ系統を全て抜いて学習させる。

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import catboost as cb
from matplotlib import pyplot as plt
import warnings
warnings.simplefilter('ignore')

In [2]:
data_dir = "F:/TARGET frontier/TXT35/"
data_name = "data"
tail = "fe3"
train_data = pd.DataFrame()
for i in range(2015,2018+1):
    res = pd.read_csv(data_dir+str(i)+data_name+tail+".csv",encoding="shift-jis",dtype={'レースID(旧)':str})
    train_data = pd.concat([train_data,res])

#test_data
test_data = pd.read_csv(data_dir+"2019data"+tail+".csv",encoding="shift-jis",dtype={'レースID(旧)':str})

In [3]:
#df_new = df.rename(columns={'A': 'a'}, 
train_data = train_data.rename(columns={'芝・ダ':'shibada'})
test_data = test_data.rename(columns={'芝・ダ':'shibada'})

In [4]:
train_data = train_data.query("馬場状態 != 1")
test_data = test_data.query("馬場状態 != 1")
train_data = train_data.query("shibada == 2")
test_data = test_data.query("shibada == 2")

In [5]:
pd.set_option('display.max_columns', 100)
train_data.head()

,レースID(新),場所,shibada,距離,クラス名,馬場状態,出走頭数,騎手,枠番,性別,年齢,キャリア,斤量,馬体重,馬体重増減,間隔,休み明け〜戦目,種牡馬,父タイプ名,所属,調教師,入線順位,確定着順,着差,単勝オッズ,複勝オッズ下限,複勝オッズ上限,複勝配当,場所距離芝ダ,前走レースID(新),前走場所,前芝・ダ,前距離,前クラス名,前走馬場状態,前走出走頭数,替,前騎手,前走枠番,前走脚質,前走斤量,前走Ave-3F,前走上り3F,前走上り3F順,前走PCI3,前走RPCI,前PCI,前好走,前走入線順位,前走確定着順,...,前走単勝オッズ,前走場所距離芝ダ,前走レースID(新)1,前走場所1,前芝・ダ1,前距離1,前クラス名1,前走馬場状態1,前走出走頭数1,替1,前騎手1,前走枠番1,前走脚質1,前走斤量1,前走Ave-3F1,前走上り3F1,前走上り3F順1,前走PCI31,前走RPCI1,前PCI1,前好走1,前走入線順位1,前走確定着順1,前走着差タイム1,前走単勝オッズ1,前走場所距離芝ダ1,前走レースID(新)2,前走場所2,前芝・ダ2,前距離2,前クラス名2,前走馬場状態2,前走出走頭数2,替2,前騎手2,前走枠番2,前走脚質2,前走斤量2,前走Ave-3F2,前走上り3F2,前走上り3F順2,前走PCI32,前走RPCI2,前PCI2,前好走2,前走入線順位2,前走確定着順2,前走着差タイム2,前走単勝オッズ2,前走場所距離芝ダ2
0,201512270605081112,2,2,1200,2,3,16,497,6,2,3,13,56.0,492,-4.0,6.0,5.0,675,8,0,363,1,1,0.0,5.4,1.9,2.7,190.0,76,201511140505031003,14,2,1400.0,2,2,15.0,0,333,2.0,3,54.0,37.2,33.9,10.0,59.53,58.2,59.7,0,8.0,8.0,...,14.6,55,201510100804011013,3.0,2.0,1400.0,2.0,3.0,18.0,0.0,435.0,7.0,2.0,53.0,35.03,34.0,11.0,54.30,51.8,53.0,0.0,8.0,8.0,0.3,10.1,202.0,201508220102011003,13.0,2.0,1200.0,1.0,3.0,16.0,0.0,101.0,2.0,2.0,54.0,34.1,34.8,3.0,47.77,44.6,48.0,0.0,1.0,1.0,0.0,15.3,44.0
1,201512270605081111,2,2,1200,2,3,16,79,6,1,3,10,54.0,466,-12.0,3.0,3.0,535,8,0,362,2,2,0.0,2.8,1.3,1.6,130.0,76,201512060704020613,1,2,1200.0,1,3,18.0,1,79,7.0,3,54.0,34.5,34.6,3.0,48.80,46.9,49.7,0,1.0,1.0,...,2.2,16,201511140303050913,19.0,2.0,1200.0,1.0,4.0,16.0,0.0,310.0,7.0,3.0,53.0,35.30,36.4,6.0,48.27,47.0,47.0,0.0,2.0,2.0,0.4,2.0,96.0,201508290102030707,13.0,2.0,1200.0,1.0,3.0,16.0,0.0,354.0,4.0,2.0,52.0,34.4,34.5,5.0,49.43,47.4,49.7,0.0,2.0,2.0,0.2,2.9,44.0
2,201512270605081106,2,2,1200,2,3,16,248,3,2,3,9,56.0,478,6.0,5.0,2.0,714,5,1,241,3,3,0.2,6.2,1.8,2.5,180.0,76,201511210805051217,3,2,1200.0,2,3,18.0,0,161,8.0,2,56.0,34.5,34.4,9.0,50.13,48.0,50.3,0,7.0,7.0,...,28.6,197,201505170803081006,3.0,2.0,1200.0,12.0,3.0,16.0,0.0,455.0,3.0,2.0,56.0,34.80,34.1,10.0,51.90,50.0,52.1,0.0,9.0,9.0,0.9,118.0,197.0,201503220602081104,2.0,2.0,1800.0,7.0,3.0,12.0,0.0,358.0,4.0,5.0,56.0,37.3,37.3,11.0,60.53,58.1,50.0,1.0,11.0,11.0,2.8,327.2,122.0
3,201512270605081108,2,2,1200,2,3,16,395,4,1,5,17,55.0,484,-4.0,2.0,4.0,108,1,1,377,4,4,0.3,6.6,1.7,2.5,0.0,76,201512130605040911,2,2,1200.0,2,2,16.0,1,395,6.0,2,53.0,34.1,34.4,3.0,48.17,46.3,49.1,0,3.0,3.0,...,19.1,118,201510240804060915,3.0,2.0,1200.0,2.0,3.0,15.0,0.0,266.0,8.0,3.0,55.0,34.30,34.7,15.0,52.50,51.2,48.8,1.0,12.0,12.0,1.4,21.5,197.0,201509200604041005,2.0,2.0,1200.0,2.0,3.0,16.0,0.0,395.0,3.0,2.0,55.0,34.4,34.4,5.0,50.80,47.1,50.0,0.0,4.0,4.0,0.2,12.5,118.0
4,201512270605081116,2,2,1200,2,3,16,449,8,1,3,15,54.0,440,-6.0,2.0,4.0,393,4,1,186,5,5,0.3,33.9,6.3,9.7,0.0,76,201512130605040904,2,2,1200.0,2,2,16.0,0,252,2.0,5,53.0,33.6,36.4,15.0,48.17,46.3,42.3,1,13.0,13.0,...,8.6,118,201510310303011107,19.0,2.0,1200.0,2.0,3.0,16.0,1.0,252.0,4.0,5.0,51.0,33.90,34.3,8.0,51.30,49.1,48.8,0.0,2.0,2.0,0.1,20.8,96.0,201509270604070708,2.0,2.0,1200.0,1.0,2.0,16.0,0.0,121.0,4.0,3.0,50.0,34.6,34.3,8.0,51.30,49.7,50.9,0.0,1.0,1.0,0.0,59.4,118.0


In [6]:
columns_name = ['クラス名','出走頭数','騎手','枠番','性別','年齢','キャリア','斤量',
'間隔','休み明け〜戦目','種牡馬','調教師','場所距離芝ダ','単勝オッズ',
'前クラス名','前走出走頭数','前騎手','前走枠番','前走脚質','前走斤量','前走Ave-3F',
'前走上り3F','前走上り3F順','前好走','前走着差タイム','前走単勝オッズ','前走場所距離芝ダ',
'前クラス名1','前走出走頭数1',
 '前走脚質1','前好走1',
 '前走着差タイム1','前走場所距離芝ダ1','前クラス名2','前走出走頭数2','前走脚質2',
 '前好走2','前走着差タイム2','前走場所距離芝ダ2']
cat_columns = ['クラス名','騎手','性別',
'種牡馬','調教師','場所距離芝ダ','前クラス名','前騎手','前走脚質','前好走','前走場所距離芝ダ',
 '前クラス名1','前走脚質1','前好走1','前走場所距離芝ダ1',
 '前クラス名2','前走脚質2','前好走2','前走場所距離芝ダ2']

In [20]:
columns_name = [ '距離', '馬場状態','クラス名', '騎手', '枠番', '性別', '年齢', 'キャリア',
       '斤量', '間隔', '休み明け〜戦目', '種牡馬', '前距離',
       '前クラス名', '前走馬場状態','前騎手', '前走枠番', '前走脚質', '前走斤量', '前走Ave-3F',
       '前走上り3F', '前好走', '前走着差タイム',"単勝オッズ",
       '前走単勝オッズ',  '出走頭数', '場所距離芝ダ', '前走場所距離芝ダ']
columns_name += ['前走着差タイム1','前走着差タイム2']
cat_columns = ['クラス名',  '騎手', '性別', '馬場状態','前走馬場状態',
               '種牡馬',
       '前クラス名', '前騎手','前走脚質', '前好走', 
       '場所距離芝ダ', '前走場所距離芝ダ']

In [21]:
use_train_columns = columns_name
cat_feature = cat_columns

In [22]:
train_x = train_data[use_train_columns]
test_x = test_data[use_train_columns]

In [23]:
test_y = test_data["入線順位"]
train_y = train_data["入線順位"]

In [24]:
#カテゴリカル変数になっている特徴量の列名を指定
categorical_features_indices = cat_feature

In [25]:
#これがないとエラー履くので注意
train_y.fillna(7,inplace=True) #無ければ10位に
test_y.fillna(7,inplace=True)

In [26]:
#上記のものは全て文字列またはint型である必要があり、floatや欠損値は変換しなければならない
for i in categorical_features_indices:
    train_x[i].fillna(9999,inplace=True)
    test_x[i].fillna(9999,inplace=True)
for i in categorical_features_indices:
    if train_x[i].dtype != 'int64':
        train_x[i] = train_x[i].astype('int64')
    if test_x[i].dtype != 'int64':
        test_x[i] = test_x[i].astype('int64')

In [27]:
scores = []
histories = []
models = []
y = pd.Series()
kf = KFold(n_splits=4,shuffle=False)
for tr_idx,va_idx in kf.split(train_x):
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    #watchlist = [(dtrain,"train"),(dvalid,"eval")]
    evals_result = {}
    model = cb.CatBoostRegressor(iterations=10000,
                                 learning_rate=0.01,
                                 depth=6,
                                 verbose=100,
                                 use_best_model=True,
                                 early_stopping_rounds=100,
                                 one_hot_max_size=100,
                                 task_type="GPU",
                                 devices='0:1'
                                 
                                )
    
    model.fit(tr_x,
              tr_y,
              cat_features=categorical_features_indices,
              eval_set=(va_x, va_y),
              plot=False,

             )
    models.append(model)
    #histories.append(evals_result)
    va_pred = model.predict(va_x)
    va_y.fillna(0.5,inplace=True)
    score = mean_squared_error(va_y,va_pred)
    score = np.sqrt(mean_squared_error(va_y,va_pred)) #rmse
    scores.append(score)
    
    #evaluationさせる
    #全部のfoldについてyに入れておいて、
    y_pred_proba = model.predict(va_x)
    y = pd.concat([y,pd.Series(y_pred_proba)])


output = train_data
output = output.assign(ya=y.values)
output["複勝配当"] = output["複勝配当"].astype("int64")
output.to_csv("C:/programs/HRP/kfeval2.csv",index=False,encoding="shift-jis")
    
print(*scores)
print(np.mean(scores))

0:	learn: 4.5174625	test: 4.5793199	best: 4.5793199 (0)	total: 21.8ms	remaining: 3m 38s
100:	learn: 3.6991691	test: 3.7990174	best: 3.7990174 (100)	total: 2.32s	remaining: 3m 47s
200:	learn: 3.5402687	test: 3.6431473	best: 3.6431473 (200)	total: 4.48s	remaining: 3m 38s
300:	learn: 3.4983931	test: 3.6017799	best: 3.6017799 (300)	total: 6.75s	remaining: 3m 37s
400:	learn: 3.4772515	test: 3.5845528	best: 3.5845528 (400)	total: 9.01s	remaining: 3m 35s
500:	learn: 3.4644420	test: 3.5766905	best: 3.5766905 (500)	total: 11.5s	remaining: 3m 37s
600:	learn: 3.4550790	test: 3.5727901	best: 3.5727901 (600)	total: 13.8s	remaining: 3m 35s
700:	learn: 3.4477778	test: 3.5702395	best: 3.5702395 (700)	total: 16.3s	remaining: 3m 36s
800:	learn: 3.4414100	test: 3.5687295	best: 3.5687295 (800)	total: 18.8s	remaining: 3m 35s
900:	learn: 3.4357160	test: 3.5676389	best: 3.5676252 (899)	total: 21.2s	remaining: 3m 34s
1000:	learn: 3.4306530	test: 3.5669397	best: 3.5669397 (1000)	total: 23.8s	remaining: 3m 33s


## 学習結果  
1.全要素入れ(パラメータチューニングなし、lr=0.01,depth=6)  
loss:3.494416  
1~3位PHR: 1.588  
1~3位WHR: 0.636  
1~6位PHR: 2.392  
回収率78%
  
2.前2,3走のレース内容についてはカット(PCI,RPCI,3F,3F順)  
loss:3.494270  
1~3位PHR: 1.589  
1~3位WHR: 0.638  
1~6位PHR: 2.392  
回収率84%  

3.所属、父タイプ、前2,3走についての替、場所、馬場状態のカット  
loss:3.494830  
1~3位PHR: 1.590  
1~3位WHR: 0.638  
1~6位PHR: 2.392  
回収率105%  

4.前2,3走についての斤量、騎手、枠番をカット、着順を全て入線のみに  
loss:3.4946046  
1~3位PHR: 1.588  
1~3位WHR: 0.638  
1~6位PHR: 2.395  
回収率104%  

5.場所距離芝ダの、元データを全てカット(場所距離芝ダを採用)  
loss:3.497740  
1~3位PHR: 1.587  
1~3位WHR: 0.637  
1~6位PHR: 2.394  
回収率109%  
6.芝・ダと前芝・ダを戻す  
loss:3.49573  
1~3位PHR: 1.587  
1~3位WHR: 0.637  
1~6位PHR: 2.394  
回収率:75%  
7.前走のPCI系統のデータをカット  
loss:3.4962  
1~3位PHR: 1.589  
1~3位WHR: 0.637  
1~6位PHR: 2.395  
回収率:83%  
8.芝ダはやっぱりカット。前2,3走の単勝オッズと、替をカット  
loss:3.4971  
1~3位PHR: 1.589  
1~3位WHR: 0.638  
1~6位PHR: 2.394  
回収率:84%  
9.馬場状態、前走馬場状態をカット  
loss:3.4975  
1~3位PHR: 1.589  
1~3位WHR: 0.639  
1~6位PHR: 2.395  
回収率:82%  
10.前走の枠番と、前好走2をカット  
loss:3.49756  
1~3位PHR: 1.588  
1~3位WHR: 0.638  
1~6位PHR: 2.395  
回収率:79%  

11.10のデータは戻しておくものとする.入線順位を全てカット(着差タイムに統一)  
loss:3.50037  
1~3位PHR: 1.588  
1~3位WHR: 0.637  
1~6位PHR: 2.398  
回収率:124%  
12.逆に、着差タイムを全てカット  
loss:3.498495  
1~3位PHR: 1.589  
1~3位WHR: 0.639  
1~6位PHR: 2.395  
回収率:85%  
  
13.11から派生。前2.3走の出走頭数をカット  
loss:3.5000  
1~3位PHR: 1.587  
1~3位WHR: 0.638  
1~6位PHR: 2.398  
回収率:122%  

14.11の状態で、2016~2018,kfold3で学習  
loss:3.5020  
1~3位PHR: 1.587  
1~3位WHR: 0.640  
1~6位PHR: 2.399  
回収率:117%  

※以降,2015~  
15.不良馬場カット、ダートのみ  
loss:3.4999  
1~3位PHR: 1.555  
1~3位WHR: 0.621   
1~6位PHR: 2.358  
回収率:137% (75%) ※()内はkfeval  

16.同芝のみ  
loss:3.5008  
1~3位PHR: 1.605  
1~3位WHR: 0.648  
1~6位PHR: 2.427  
回収率:274.44% (102%) ←的中3回なので、あまり実用的ではない  

17.11から派生。芝ダ、前芝ダを戻す  
loss:3.49814  
1~3位PHR: 1.587  
1~3位WHR: 0.637  
1~6位PHR: 2.392  
回収率:112% (92%)  

18.馬体重、馬体重増減をカット(芝ダどっちも)  
loss:3.4983  
1~3位PHR: 1.584    
1~3位WHR: 0.637  
1~6位PHR: 2.397  
回収率:120% (102%)  

In [28]:
test_length = len(test_data)
predict = np.array([0.0]*test_length)
for i in range(4):
    y_pred_proba = models[i].predict(test_x)
    predict += y_pred_proba

predict /= 4

In [29]:
#y = pd.DataFrame(predict,columns=["y'"])
output = test_data #ここも書き換えること

output["ya"] = predict
output.head()
output["複勝配当"] = output["複勝配当"].astype("int64")
output.to_csv("C:/programs/HRP/neweval11.csv",index=False,encoding="shift-jis")

In [31]:
fscore = model.get_feature_importance()

In [32]:
jscore = []
for i,j in zip(use_train_columns,fscore):
    jscore.append((i,np.round(j,decimals=3)))
    

In [33]:
from operator import itemgetter
jscore.sort(reverse=True,key=itemgetter(1))

In [34]:
jscore

[('単勝オッズ', 61.421),
 ('出走頭数', 22.211),
 ('前走上り3F順', 2.515),
 ('間隔', 1.685),
 ('前走脚質', 1.511),
 ('休み明け〜戦目', 1.058),
 ('前走脚質1', 0.928),
 ('前走脚質2', 0.704),
 ('前走着差タイム', 0.695),
 ('場所距離芝ダ', 0.64),
 ('種牡馬', 0.597),
 ('調教師', 0.549),
 ('前走場所距離芝ダ', 0.526),
 ('前走場所距離芝ダ1', 0.5),
 ('前走場所距離芝ダ2', 0.457),
 ('前好走1', 0.394),
 ('前クラス名2', 0.379),
 ('斤量', 0.377),
 ('騎手', 0.369),
 ('前走着差タイム1', 0.358),
 ('前走Ave-3F', 0.342),
 ('枠番', 0.252),
 ('前クラス名', 0.2),
 ('前好走', 0.172),
 ('前好走2', 0.168),
 ('年齢', 0.142),
 ('キャリア', 0.131),
 ('クラス名', 0.103),
 ('性別', 0.099),
 ('前走着差タイム2', 0.097),
 ('馬体重増減', 0.09),
 ('前走斤量', 0.089),
 ('前走単勝オッズ', 0.073),
 ('前騎手', 0.046),
 ('前走上り3F', 0.046),
 ('前クラス名1', 0.03),
 ('前走出走頭数1', 0.026),
 ('前走出走頭数', 0.018),
 ('馬体重', 0.0),
 ('前走枠番', 0.0),
 ('前走出走頭数2', 0.0)]

In [ ]:
slen = 20
label = []
weight = []
for i,j in fscore[:slen]:
    label.append(i)
    weight.append(j)

plt.rcdefaults() #横棒グラフモード
fig, ax = plt.subplots()
y_pos = np.arange(slen)
ax.barh(y_pos, weight,align='center')
ax.set_yticks(y_pos)
ax.set_yticklabels(label)
ax.invert_yaxis()
ax.set_xlabel('label')
ax.set_title('xgboost特徴量')
plt.show()
    

In [ ]:
#save models
import pickle
with open('xgmachina_ver0.0.7.pickle', mode='wb') as fp:
    pickle.dump(models, fp)

In [ ]:
#import models
"""import pickle
with open('xgmachina_ver0.0.6.pickle', mode='rb') as fp:
    models = pickle.load(fp)"""